In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dypy.plotting import Mapfigure
import kedro.framework.context

plt.rcParams.update({'font.size': 20, 'figure.figsize': (16,9)})

# Load data

In [ ]:
df_fires = context.catalog.load("fire_data_cleansed_stations")
df_stations = context.catalog.load("foehn_stations")

# Plot fire and foehn stations

In [ ]:
domain = [5.9, 10.6, 45.65, 48]
m = Mapfigure(domain=domain)
fig = plt.figure()

# Plot fires
plt.plot(df_fires["longitude"], df_fires["latitude"], "x")

# Plot foehn stations
plt.plot(df_stations["longitude"], df_stations["latitude"], "ro", markersize=8)

# Format legend
blue_cross = mlines.Line2D([], [], color='blue', marker='x', linestyle='None',
                          markersize=10, label='Forest fire')
red_oval = mlines.Line2D([], [], markeredgecolor='red',color ="red", marker='o', linestyle='None',
                          markersize=8, label="Foehn oberservation station")
plt.legend(handles=[blue_cross, red_oval], loc=2)

# Move labels to non-overlapping position
for i in range(len(df_stations)):
    if i in {11,0,35,27, 8,6,10,25}:
        plt.annotate(df_stations.loc[i, "name"],
                     (df_stations.loc[i,"longitude"], df_stations.loc[i,"latitude"]),
                      # distance from text to points (x,y)
                     textcoords="offset points",
                     xytext=(0,-17),
                     ha="center",
                     color="r",
                     backgroundcolor="w",
                     fontsize=11
                    )
    elif i in {22,17,24}:
        plt.annotate(df_stations.loc[i, "name"],
                     (df_stations.loc[i,"longitude"], df_stations.loc[i,"latitude"]),
                      # distance from text to points (x,y)
                     textcoords="offset points",
                     xytext=(11,-3),
                     ha="left",
                     color="r",
                     backgroundcolor="w",
                     fontsize=11
                    )
    elif i in {7,21,33,32,15}:
        plt.annotate(df_stations.loc[i, "name"],
                     (df_stations.loc[i,"longitude"], df_stations.loc[i,"latitude"]),
                      # distance from text to points (x,y)
                     textcoords="offset points",
                     xytext=(-8,-3),
                     ha="right",
                     color="r",
                     backgroundcolor="w",
                     fontsize=11
                    )
    else:
        plt.annotate(df_stations.loc[i, "name"],
                     (df_stations.loc[i,"longitude"], df_stations.loc[i,"latitude"]),
                      # distance from text to points (x,y)
                     textcoords="offset points",
                     xytext=(0,11),
                     ha="center",
                     color="r",
                     backgroundcolor="w",
                     fontsize=11
                    )
    
m.drawmap(nbrem=1, nbrep=1)

# Save figure
fig_name = f'/data/08_reporting/ForestFiresFoehnStation.pdf'
plt.savefig(fig_name, bbox_inches='tight', dpi=200)
print(f"Saved figure at: {fig_name}")

# Plot ellipsoids around each stations

In [ ]:
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
import matplotlib.lines as mlines

def geodesic_point_expansion(lat, lon, km):
    """
    Expand a point to a circle
    @param lat, lon: Coordinates
    @param km: Size of circle in km
    @returns: List of tuples with coords
    """
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        pyproj.Proj('+proj=longlat +datum=WGS84'))
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:]

In [ ]:
domain = [5.9, 10.6, 45.65, 48]
m = Mapfigure(domain=domain)
fig = plt.figure()

# Plot fires
plt.scatter(df_fires["longitude"], df_fires["latitude"], marker="x")

# Plot ellipsoids for each station
for i in range(len(df_stations["longitude"])):
    ellipse = geodesic_point_expansion(df_stations.loc[i, "latitude"], df_stations.loc[i, "longitude"], 20)
    x= [point[0] for point in ellipse]
    y= [point[1] for point in ellipse]
    plt.plot(x, y, "-", color="r", linewidth=1)

# Format legend
blue_cross = mlines.Line2D([], [], color='blue', marker='x', linestyle='None',
                          markersize=10, label='Forest fire')
red_oval = mlines.Line2D([], [], markeredgecolor='red',color ="white", marker='o', linestyle='None',
                          markersize=15, label="Foehn oberservation station radius")
plt.legend(handles=[blue_cross, red_oval], loc=2)
    
m.drawmap(nbrem=1, nbrep=1)
plt.gca().set_aspect("equal")

# Save figure
fig_name = f'/data/08_reporting/ForestFiresFoehnStationRadius.pdf'
plt.savefig(fig_name, bbox_inches='tight', dpi=200)
print(f"Saved figure at: {fig_name}")